# Cross Check up for hdf5 vs. hdfs parquets

- tba 


In [29]:
# basic packages
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import glob
#import databricks.koalas as ks

# the default `sequence` option merges all partitions into a single catastrophic one : what?
#ks.set_option('compute.default_index_type', 'distributed') 


## Read data 

In [30]:
workdir = !pwd
print(workdir[0])

/home/shong/work/illustris/script/snapshot


#### Original HDF5 Path

In [31]:
h5dir = '/mnt/data/shong/tempdata/tng50snap84/raw/'

In [32]:
h5list = sorted(glob.glob(h5dir+'snap*.hdf5'))

In [33]:
numh5list = len(h5list)
print(numh5list)

680


In [34]:
h5list[:5]

['/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.0.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.1.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.10.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.100.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.101.hdf5']

In [35]:
h5list[-5:]

['/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.95.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.96.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.97.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.98.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.99.hdf5']

> Apparently, no issue in original files. 

#### HDFS Parquet Path

In [36]:
alldmfiles = !hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/dm/*

In [37]:
alldmfiles[:3]

['857.4 M  2.5 G  /common/data/illustris/tng50/snapshot84/dm/snap_084.0_dm.parquet.snappy',
 '837.9 M  2.5 G  /common/data/illustris/tng50/snapshot84/dm/snap_084.100_dm.parquet.snappy',
 '820.4 M  2.4 G  /common/data/illustris/tng50/snapshot84/dm/snap_084.101_dm.parquet.snappy']

In [38]:
len(alldmfiles)

680

In [39]:
allgasfiles = !hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/gas/*

In [40]:
allgasfiles[:3]

['2.3 G  7.0 G  /common/data/illustris/tng50/snapshot84/gas/snap_084.0_gas.parquet.snappy',
 '2.4 G  7.3 G  /common/data/illustris/tng50/snapshot84/gas/snap_084.100_gas.parquet.snappy',
 '2.4 G  7.2 G  /common/data/illustris/tng50/snapshot84/gas/snap_084.101_gas.parquet.snappy']

In [41]:
len(allgasfiles)

680

In [42]:
allstarfiles = !hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/star/*

In [43]:
allstarfiles[:3]

['353.9 M  1.0 G     /common/data/illustris/tng50/snapshot84/star/snap_084.0_star.parquet.snappy',
 '272.9 M  818.8 M   /common/data/illustris/tng50/snapshot84/star/snap_084.100_star.parquet.snappy',
 '350.9 M  1.0 G     /common/data/illustris/tng50/snapshot84/star/snap_084.101_star.parquet.snappy']

In [44]:
len(allstarfiles)

680

In [45]:
allbhfiles = !hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/bh/*

In [46]:
allbhfiles[:3]

['52.8 K   158.3 K  /common/data/illustris/tng50/snapshot84/bh/snap_084.0_bh.parquet.snappy',
 '59.1 K   177.2 K  /common/data/illustris/tng50/snapshot84/bh/snap_084.100_bh.parquet.snappy',
 '21.4 K   64.1 K   /common/data/illustris/tng50/snapshot84/bh/snap_084.101_bh.parquet.snappy']

In [47]:
len(allbhfiles)

678

### Cross Check 

In [48]:
import re

# Step 1: Extract unique identifiers from h5list
h5_ids = set(
    re.search(r'snap_084\.\d+', path).group()
    for path in h5list
)

# Step 2: Extract from alldmfiles (skip size info, extract only filename)
dm_ids = set(
    re.search(r'snap_084\.\d+', path).group()
    for path in alldmfiles
)

In [49]:
# Step 3: Identify missing ID(s)
missing_ids = h5_ids - dm_ids

# Print results
print("Missing file(s):", missing_ids)

Missing file(s): set()


In [50]:
star_ids = set(
    re.search(r'snap_084\.\d+', path).group()
    for path in allstarfiles
)

# Step 3: Identify missing ID(s)
missing_ids = h5_ids - star_ids

# Print results
print("Missing file(s):", missing_ids)

Missing file(s): set()


In [51]:
gas_ids = set(
    re.search(r'snap_084\.\d+', path).group()
    for path in allgasfiles
)

# Step 3: Identify missing ID(s)
missing_ids = h5_ids - gas_ids

# Print results
print("Missing file(s):", missing_ids)

Missing file(s): set()


In [52]:
bh_ids = set(
    re.search(r'snap_084\.\d+', path).group()
    for path in allbhfiles
)

# Step 3: Identify missing ID(s)
missing_ids = h5_ids - bh_ids

# Print results
print("Missing file(s):", missing_ids)

Missing file(s): {'snap_084.295', 'snap_084.520'}
